# DNABERT 모델 불러오기
### (모델 사용 환경 설정 포함)

In [1]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 GPU 사용 가능

True


In [2]:
# github 환경 가져오기
!git clone https://github.com/MAGICS-LAB/DNABERT_2.git
%cd DNABERT_2

Cloning into 'DNABERT_2'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 98 (delta 18), reused 24 (delta 15), pack-reused 64 (from 1)
Receiving objects: 100% (98/98), 870.88 KiB | 29.03 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/DNABERT_2


In [3]:
# DNABERT-2 실행에 필요한 모든 패키지를 포함 (필수)

!python3 -m pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.

In [ ]:
# 설치된 transformer 버전 확인
import transformers
print(transformers.__version__)

4.29.2


In [ ]:
!pip list | grep transformers
!pip list | grep torch

# 충돌 에러가 떴지만 주요 패키지가 제대로 설치되어 있으므로 넘어갔다.

sentence-transformers              3.2.1
transformers                       4.29.2
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchsummary                       1.5.1
torchvision                        0.20.1+cu121


In [4]:
# DNABERT-2 모델 및 토크나이저 로드
# transformer 버전 4.28 초과 시 실행하는 코드
from transformers.models.bert.configuration_bert import BertConfig
from transformers import AutoTokenizer, AutoModel

# DNABERT-2 설정 로드
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")

# DNABERT-2 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

bert_layers.py:   0%|          | 0.00/40.7k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_layers.py
- bert_padding.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:446: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [ ]:
# 예시 (가져온 tokenizer와 model이 제대로 실행되는지 확인)
# DNA 시퀀스 임베딩 계산

import torch

# 예제 DNA 시퀀스
dna = "ACGTAGCATCGACTATCATCAGACACTGGTATTGACGTACGAGCATCTGTTAGC"

# 토큰화
inputs = tokenizer(dna, return_tensors="pt")["input_ids"]

# 모델 실행
hidden_states = model(inputs)[0]  # [batch_size, sequence_length, hidden_size]

# Mean Pooling (평균 풀링)
embedding_mean = torch.mean(hidden_states[0], dim=0)
print("Mean Pooling Shape:", embedding_mean.shape)  # (768,)

# Max Pooling (최대값 풀링)
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print("Max Pooling Shape:", embedding_max.shape)  # (768,)

Mean Pooling Shape: torch.Size([768])
Max Pooling Shape: torch.Size([768])


# Fine-tuning

In [5]:
%cd finetune

/content/DNABERT_2/finetune


In [6]:
# Colab용 환경 설정
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "./INPUT"  # 설정한 input 데이터 경로에 train, dev, test 모두 존재하는지 확인

dev.csv  test.csv  train.csv


In [7]:
# 설정
DATA_PATH = "'./INPUT'"
OUTPUT_DIR = "'./OUTPUT'" # checkpoint 모델 저장될 경로
MAX_LENGTH = 102  # 최대 DNA 서열 길이 * 0.25
LR = 3e-5  # 학습률 설정 

### 5 epoch 학습

In [10]:
# Training with DataParallel
!DISABLE_TQDM=1 python train.py \
    --model_name_or_path zhihan1996/DNABERT-2-117M \
    --data_path $DATA_PATH \
    --kmer -1 \
    --run_name DNABERT2_$DATA_PATH \
    --model_max_length $MAX_LENGTH \
    --per_device_train_batch_size 64 \
    --per_device_eval_batch_size 128 \
    --gradient_accumulation_steps 2 \
    --learning_rate $LR \
    --num_train_epochs 5 \
    --fp16 \
    --save_steps 2500 \
    --output_dir $OUTPUT_DIR \
    --evaluation_strategy steps \
    --eval_steps 2500 \
    --warmup_steps 50 \
    --logging_steps 500 \
    --overwrite_output_dir True \
    --log_level warning \
    --find_unused_parameters False


2024-12-13 17:45:53.752311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 17:45:53.773590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 17:45:53.780064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 17:45:54.948018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, us

### Fine-tuning 관련 자세한 사항은 DNABERT 깃허브 참조